In [7]:
import pdb
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.optimize import minimize
import random
from pymoo.operators.selection.tournament import TournamentSelection
from problem import MyProblem
from mutation import MyMutation
from crossover import TraceCrossover
from mock_tools import MockEncoder, MockDeclare4Py
from sampling import MySampling
from callback import UpdatePopulationCallback
from growingGA import GrowingNSGA2, PreSurvivalNSGA2
import numpy as np
from tools import Tools
from survival import MySurvival
from pymoo.operators.crossover.sbx import SBX
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.ProcessModels.DeclareModel import DeclareModelTemplate
from Declare4Py.D4PyEventLog import D4PyEventLog
import warnings
import random
from datetime import datetime, timedelta
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
import os
import pandas as pd
warnings.filterwarnings("ignore", ".*feasible.*")

# n_events = 5-10-15-20
# n_activities = 5-10-15-20-25-30

activities_name = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T"]
variable_boundaries = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
timestamps = Tools.generate_random_timestamps(len(activities_name))
case_concept_name = ['1'] * len(activities_name)

data = {
    'case:concept:name': case_concept_name,
    'concept:name': activities_name,
    'timestamp': timestamps,
}

dataframe = pd.DataFrame(data)


dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'])


encoder = MockEncoder(activities_name)
declare = DeclareModel().parse_from_file("base_model.decl")
model_constraints = declare.get_decl_model_constraints()

event_log = D4PyEventLog()
event_log.log = dataframe

# TODO make some info
event_log.timestamp_key = "timestamp"
event_log.activity_key = "concept:name"

event_log.to_eventlog()

basic_checker = MPDeclareAnalyzer(log=event_log, declare_model=declare, consider_vacuity=False)
conf_check_res: MPDeclareResultsBrowser = basic_checker.run()

print(conf_check_res.get_metric(trace_id=0, metric="num_activations"))
print("-------------------------------------------")

d4py = MockDeclare4Py()

[None, None, None, 1, 1, 1]
-------------------------------------------


In [8]:

# create random pop
n_traces = 10
trace_length = 20

initial_population = [[random.choice(activities_name) for _ in range(trace_length)] for _ in range(n_traces)]
initial_encoded_pop = [encoder.encode(trace) for trace in initial_population]
features_range = Tools.calculate_feature_range(initial_encoded_pop, variable_boundaries)
lower_bounds = [x[0] for x in features_range]
upper_bounds = [x[1] for x in features_range]

print(initial_population)
print(initial_encoded_pop)

# expanded_pop = Tools.expand_population(initial_encoded_pop, 50, features_range)

[['S', 'D', 'K', 'A', 'A', 'A', 'R', 'A', 'M', 'G', 'N', 'A', 'Q', 'H', 'O', 'P', 'R', 'H', 'L', 'H'], ['H', 'O', 'J', 'A', 'N', 'R', 'D', 'F', 'J', 'D', 'K', 'Q', 'N', 'Q', 'G', 'J', 'J', 'S', 'P', 'Q'], ['M', 'S', 'B', 'P', 'H', 'M', 'N', 'F', 'L', 'R', 'L', 'C', 'O', 'Q', 'D', 'F', 'Q', 'M', 'L', 'P'], ['A', 'P', 'B', 'J', 'T', 'S', 'S', 'M', 'F', 'F', 'Q', 'H', 'A', 'G', 'R', 'R', 'H', 'M', 'Q', 'L'], ['S', 'L', 'O', 'I', 'R', 'T', 'A', 'M', 'Q', 'E', 'Q', 'R', 'G', 'N', 'B', 'P', 'L', 'S', 'R', 'G'], ['Q', 'N', 'P', 'L', 'N', 'L', 'A', 'R', 'R', 'T', 'T', 'K', 'O', 'T', 'A', 'H', 'F', 'R', 'S', 'F'], ['C', 'R', 'I', 'B', 'C', 'C', 'A', 'O', 'A', 'I', 'H', 'I', 'D', 'T', 'F', 'L', 'J', 'C', 'F', 'F'], ['I', 'Q', 'F', 'I', 'J', 'O', 'K', 'P', 'P', 'D', 'A', 'J', 'M', 'K', 'N', 'G', 'I', 'D', 'I', 'Q'], ['G', 'T', 'N', 'A', 'H', 'A', 'M', 'E', 'B', 'F', 'O', 'Q', 'N', 'R', 'H', 'Q', 'O', 'H', 'Q', 'A'], ['M', 'S', 'K', 'N', 'B', 'J', 'E', 'G', 'B', 'J', 'C', 'C', 'J', 'J', 'F', 'N', 

In [9]:
from terminator import MyTermination

# instantiate problem, callback, mutation, sampling, crossover and survival
problem = MyProblem(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)

callback = UpdatePopulationCallback(problem)
mutation = MyMutation(features_range)
crossover = TraceCrossover(variable_boundaries=variable_boundaries)
sampling = MySampling(initial_encoded_pop)
survival = MySurvival(n_children_survive=50)

algorithm = NSGA2(
    pop_size=500,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=callback,
    eliminate_duplicates=False,
)

termination = MyTermination(50)
# TODO check if we can set a pop final size as a terminator param
result = minimize(problem, algorithm, termination=termination, seed=1, verbose=True)

traces = [individual.X.tolist() for individual in result.pop]

for i, trace in enumerate(traces):
    print(f"Trace {i + 1}: {trace}")

print("Best Solution Encoded:", result.X)
print("Best Solution Decoded:", [encoder.decode(solution) for solution in result.X])
print("Objective Values:", result.F)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       10 |      1 |  0.000000E+00 |  0.7000000000 |             - |             -
Trace 1: [8, 16, 5, 8, 9, 14, 10, 15, 15, 3, 0, 9, 12, 10, 13, 6, 8, 3, 8, 16]
Trace 2: [0, 15, 1, 9, 19, 18, 18, 12, 5, 5, 16, 7, 0, 6, 17, 17, 7, 12, 16, 11]
Trace 3: [16, 13, 15, 11, 13, 11, 0, 17, 17, 19, 19, 10, 14, 19, 0, 7, 5, 17, 18, 5]
Trace 4: [18, 3, 10, 0, 0, 0, 17, 0, 12, 6, 13, 0, 16, 7, 14, 15, 17, 7, 11, 7]
Trace 5: [7, 14, 9, 0, 13, 17, 3, 5, 9, 3, 10, 16, 13, 16, 6, 9, 9, 18, 15, 16]
Trace 6: [6, 19, 13, 0, 7, 0, 12, 4, 1, 5, 14, 16, 13, 17, 7, 16, 14, 7, 16, 0]
Trace 7: [18, 11, 14, 8, 17, 19, 0, 12, 16, 4, 16, 17, 6, 13, 1, 15, 11, 18, 17, 6]
Trace 8: [12, 18, 1, 15, 7, 12, 13, 5, 11, 17, 11, 2, 14, 16, 3, 5, 16, 12, 11, 15]
Trace 9: [2, 17, 8, 1, 2, 2, 0, 14, 0, 8, 7, 8, 3, 19, 5, 11, 9, 2, 5, 5]
Trace 10: [12, 18, 10, 13, 1, 9, 4, 6, 1, 9, 2, 2, 9, 9, 5, 13, 18, 8, 4, 0]
Best Solution 